In [ ]:
import os, sys, glob, scipy
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import nilearn, nibabel, nltools
from nltools.data import Brain_Data
from nilearn.plotting import plot_glass_brain, plot_stat_map, plot_img, plot_roi
from nilearn.image import new_img_like
from nistats.reporting import get_clusters_table
from nibabel.nifti1 import Nifti1Image
import nibabel as nib

## Functions

In [ ]:
def get_clusters(image, stat_threshold = 0, extent_threshold = 0, sort_by_size = False, include_peaks_only = True):
    
    # Compute negative image
    neg_image_arr = -(image.get_data().copy())
    neg_image = Nifti1Image(neg_image_arr, image.affine.copy())
    
    # Find clusters
    clusters = get_clusters_table(image, stat_threshold, cluster_threshold = extent_threshold)
    clusters = clusters.rename(columns = {'Cluster ID':'ID', 'Cluster Size (mm3)':'size',
                                          'Peak Stat':'peak_value'})
    neg_clusters = get_clusters_table(neg_image, stat_threshold, cluster_threshold = extent_threshold)
    neg_clusters = neg_clusters.rename(columns = {'Cluster ID':'ID', 'Cluster Size (mm3)':'size',
                                          'Peak Stat':'peak_value'})
    neg_clusters['peak_value'] = -neg_clusters['peak_value']
    
    # Append pos and neg
    clusters = clusters.append(neg_clusters)
    clusters['abs_peak_value'] = clusters['peak_value'].apply(np.abs)
    clusters = clusters.sort_values(by='abs_peak_value', ascending = False)
    
    # Get peaks
    if include_peaks_only:
        peaks = clusters.copy()
        peaks = peaks.loc[peaks['ID'].apply(lambda x: not str(x)[-1].isalpha()),:].reset_index(drop=True)
        if sort_by_size:
            peaks = peaks.sort_values(by = 'size', ascending = False).reset_index(drop=True)
    
    return clusters, peaks

In [ ]:
def xyz_to_ijk(MNI, image):
    return list(np.linalg.inv(image.affine[:3,:3]).dot(MNI-image.affine[:3,3]).astype(int))

In [ ]:
def expand_cluster(image, cluster_peak_MNI, verbose = True):
    
    # Find IJK of peak
    i,j,k = xyz_to_ijk(cluster_peak_MNI,image)
    cluster_peak_IJK = [i,j,k]
    if verbose: print('MNI: %s, IJK: %s'%(cluster_peak_MNI, cluster_peak_IJK))
    
    # Double check that peak value is correct
    peakval = image.get_data()[i][j][k]
    if verbose: print('Peak value extracted from image data array: %f'%peakval)
    
    # Binarize image
    binarized = (image.get_data() != 0).astype(int)
        
    # Label each cluster with a different number, reserve 0 for empty voxels
    conn_mat = np.zeros((3, 3, 3), int)  # 6-connectivity, aka NN1 or "faces"
    conn_mat[1, 1, :] = 1
    conn_mat[1, :, 1] = 1
    conn_mat[:, 1, 1] = 1
    label_map = scipy.ndimage.measurements.label(binarized, conn_mat)[0]
    clust_image = nibabel.Nifti1Image(label_map, affine=image.affine)
    
    # Find voxels with same label as cluster peak
    cluster_label = clust_image.get_data()[i,j,k]
    cluster_ROI = (clust_image.get_data() == cluster_label).astype(int)
    ROI_mask = nibabel.Nifti1Image(cluster_ROI, affine = image.affine)
    if verbose: print('Cluster ROI located')
    if verbose: print('Cluster size as extracted from image data array: %i voxels = %i mm^3'%(
        sum(cluster_ROI.flatten()),sum(cluster_ROI.flatten())*8))
    
    return cluster_peak_IJK, cluster_ROI, ROI_mask

In [ ]:
def apply_extent_threshold(fname, extent_threshold, verbose = False):
    
    print('Loading stat map...')
    beta_map = nilearn.image.load_img(fname)
    beta_map_arr = beta_map.get_data()

    print('Preparing thresholded mask...')
    all_cluster_mask_arr = beta_map_arr.copy().astype(int)
    all_cluster_mask_arr[:] = 0

    print('Counting clusters...')
    clusters, peaks = get_clusters(beta_map, threshold = 0,
                               extent_threshold = extent_threshold, include_peaks_only = True)
    cluster_list = peaks['ID'].unique()
    print('%i positive and %i negative clusters found.'%(
        peaks.loc[peaks['peak_value']>0,:].shape[0],
        peaks.loc[peaks['peak_value']<0,:].shape[0]))

    for ID in cluster_list:
        cluster_info = peaks.query('ID == @ID')
        cluster_peak_MNI = list(cluster_info.iloc[0].loc[['X','Y','Z']].values.flatten())
        cluster_peak_IJK, cluster_voxels, cluster_mask = expand_cluster(
            beta_map, cluster_peak_MNI, verbose = False)
        cluster_mask_arr = cluster_mask.get_data().astype(int)
        if verbose:
            print('Cluster %i, nvox = %i.'%(ID,np.sum(cluster_mask_arr[:])))
        else:
            print(ID, end = ',')
        all_cluster_mask_arr += cluster_mask_arr

    # Mask with extent-thresholded data
    print('Masking...')
    masked_beta_array = np.multiply(beta_map_arr, all_cluster_mask_arr)
    beta_map_extthres = Nifti1Image(masked_beta_array, beta_map.affine.copy())

    # Save
    print('Writing...')
    #     ext = '.nii' + fname.split('.nii')[1]
    ext = '.nii'
    out_fname = fname.split('.nii')[0] + '_ext-thr-%i'%extent_threshold + ext
    beta_map_extthres.to_filename(out_fname)
    print('Saved to %s.'%out_fname)

## Files

In [ ]:
base_dir = os.path.realpath('../../..')
print(base_dir)
results_dir = os.path.realpath(base_dir + '/Results/voxelwise_ISRSA/nifti')
results_dir

## Select run and model

In [ ]:
models = ['ideology']
predictors = ['scale(ideology_similarity)']
thresholds = ['fdr-0.05']
extent_threshold = 5

In [ ]:
clust_info = pd.DataFrame(columns = 
                          ['run','model','predictor','threshold','exists','n_clust','peak_val','peak_size'])
for run in [1,2,3]:
    print('Run %i'%run)
    for model in models:
        print('Model %s'%model, end = '; ')
        for predictor in predictors:
            for threshold in thresholds:
                fnames = glob.glob(results_dir + '/run-%i_model-%s/dyad_ISC_regressor-%s_beta-thr-pval-%s.nii.gz'%(
                                    run, model, predictor, threshold))
                if len(fnames) > 0:
                    if len(fnames) > 1:
                        print("Warning: more than 1 filename match!")
                    fname = fnames[0]
                    img = nib.load(fname)
                    clusters, peaks = get_clusters(img, extent_threshold = extent_threshold, sort_by_size = False)
                    n_clust = peaks.shape[0]
                    exist = True
                    if n_clust > 0:
                        peak_val, peak_size = [peaks.iloc[0,:][i] for i in ['peak_value','size']]
                    else:
                        peak_val, peak_size = [np.nan, np.nan]
                else:
                    n_clust = np.nan
                    exist = False
                to_append = pd.DataFrame([[run,model,predictor,threshold,exist,n_clust,peak_val,peak_size]],
                                         columns = clust_info.columns)
                clust_info = clust_info.append(to_append).reset_index(drop=True)
        print('')

In [ ]:
clust_info

## Pull out one run/model

In [ ]:
clust_info = pd.DataFrame(columns = 
                          ['run','model','predictor','threshold','exists','n_clust','peak_val','peak_size'])
run = 3
model = 'ideology'
threshold = 'fdr-0.05'
predictors = ['scale(ideology_similarity)']

for predictor in predictors:
    fnames = glob.glob(results_dir + '/run-%i_model-%s/dyad_ISC_regressor-%s_beta-thr-pval-%s.nii'%(
                        run, model, predictor, threshold))
    if len(fnames) > 0:
        if len(fnames) > 1:
            print("Warning: more than 1 filename match!")
        fname = fnames[0]
        img = nib.load(fname)
        plot_stat_map(img, title = predictor)
        plt.show()
        clusters, peaks = get_clusters(img)
        n_clust = peaks.shape[0]
        display(peaks)